<a href="https://colab.research.google.com/github/Wasabi-Bobby/MachineLearningHomework/blob/master/LogisticRegressionWithNumpySoftmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from keras.datasets import mnist

(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

train_labels = train_labels_original.reshape((60000, 1))

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

test_labels = test_labels_original.reshape((10000, 1))

Using TensorFlow backend.


In [0]:
import numpy as np

a = np.random.permutation(10)
arg_max = np.argmax(a)
print(a)
print(arg_max)
print(a[arg_max])

[3 2 6 5 4 0 7 1 8 9]
9
9


In [0]:
import numpy as np
import math

class Layer:
  
  def __init__(self, number_classifiers, output_dim):
    self.number_classifiers = 10
    self.output_dim = output_dim
    self.loss_points = np.array([])
    self.mini_batch_size = 6000
    self.epoch = 10
    # Change this to either MSE or BCE or SMCE
    self.option = "SMCE"
    # Rate of learning
    # Be careful about setting this too high, turns out I get an overflow error due to my bias rapidly increasing at an alarming rate
    # Don't know if I should cap the bias or not, gotta ask the professor about this one
    self.rate = .1
    
    self.s = np.random.permutation(len(train_labels))

    # Added a division of sqrt(input) for initializaing weights
    # This is known as the Xavier Initialization and has less variance
    self.weight = np.random.randn(number_classifiers, 28 * 28) / math.sqrt(10)
    self.weight = np.array(self.weight, dtype=np.float64)
    self.bias = np.random.random_sample(10)
    self.bias = np.array(self.bias, dtype=np.float64)
    
  def test(self):
    #Initializing variables
    correct_tally = 0
    a_matrix = np.zeros(10)
    
    for i in range((int)(len(test_labels))):
      current_feature = test_images[i]
      #Resetting A matrix
      a_matrix.fill(0)
      z_sum = 0
      z_matrix = np.zeros(10)
      a_matrix = np.zeros(10)
      for j in range(len(self.weight)):
        current_weight  = self.weight[j]
        z = current_feature.T.dot(current_weight)  + self.bias[j]
        z_matrix[j] = z
        z_sum += math.e**z
      for j in range(len(self.weight)):
        a = softmax(z_matrix[j], z_sum)
        a_matrix[j] = a
      maximum_arg = a_matrix.argmax()
      if maximum_arg == test_labels[i]:
        correct_tally += 1
        
    if correct_tally == 0:
      print("The classifiers managed to have an accurate guess rate of : 0% ")
    else:
      print("The classifiers managed to have an accurate guess rate of : " + str(correct_tally / len(test_labels) * 100))
  

  def forward_prop(self):
    for epoch_index in range(self.epoch):
      for i in range(self.mini_batch_size):
        current_index   = epoch_index * self.mini_batch_size + i
        current_feature = train_images[self.s[current_index]]
        current_label   = train_labels[self.s[current_index]]
        
        kronecker_delta = np.zeros((10,10))
        #print(kronecker_delta)
        for j in range(10):
          kronecker_delta[j][j] = 1
        z_sum = 0.0
        z_matrix = np.zeros(10)
        a_matrix = np.zeros(10)
        y_matrix = np.zeros(10)
        y_matrix[current_label] = 1
        for j in range(len(self.weight)):
          current_weight  = self.weight[j]
          z = current_feature.T.dot(current_weight)  + self.bias[j]
          z_matrix[j] = z
          z_sum += math.e**z
        for j in range(len(self.weight)):
          a = softmax(z_matrix[j], z_sum)
          a_matrix[j] = a
        self.backward_prop_categorical(a_matrix, kronecker_delta, current_feature, y_matrix)
        
  def backward_prop_categorical(self, a_matrix, kronecker_delta, current_feature, y_matrix):
    for i in range(len(self.weight)):
      matrix_sum = 0
      
      for j in range(len(self.weight)):
        matrix_sum += y_matrix[j] * (a_matrix[i] - kronecker_delta[j][i])
        #print(matrix_sum)
      self.weight[i] = self.weight[i] - self.rate * matrix_sum * current_feature
      #print(self.bias[i])
      self.bias[i]   = self.bias[i] - self.rate * matrix_sum
      if self.bias[i] > 10 or self.bias[i] < -10:
        print(self.bias[i])
      
def softmax(z, z_total):
  
  return math.e**z / z_total

In [32]:
new_layer = Layer(10, 1)
new_layer.forward_prop()
new_layer.test()

[4.21011141e-17 1.45721081e-29 1.36815688e-16 1.47383539e-11
 1.07716378e-22 6.03993402e-18 2.81453870e-32 1.00000000e+00
 5.62856824e-17 2.96672819e-14]
0.9999999999852318
[1.67485158e-06 2.27717426e-11 9.99998131e-01 5.21249321e-09
 1.88993388e-32 1.62115241e-10 4.99876762e-08 1.65524826e-41
 1.38912506e-07 2.41479167e-28]
0.9999981308508543
[1.77080489e-18 9.99993222e-01 6.37536564e-06 2.31739151e-07
 5.72566878e-12 5.00414820e-08 8.30877290e-11 5.75227212e-08
 6.34400606e-08 4.13753054e-11]
0.9999932217607557
[1.00000000e+00 1.64594838e-28 1.08875986e-12 1.15014476e-19
 4.28909482e-21 2.76045639e-16 5.07587336e-12 2.39426772e-16
 7.81920150e-18 1.50090586e-16]
0.9999999999938346
[7.86039329e-12 2.54822504e-18 1.52804100e-06 3.38234014e-16
 9.99608175e-01 1.79723227e-12 5.49476233e-11 3.89066245e-04
 2.27772775e-07 1.00332653e-06]
0.9996081745497195
[2.50532123e-22 9.99999975e-01 1.95553580e-09 2.28152793e-09
 9.74138049e-16 2.00511820e-12 1.50858923e-16 1.31968815e-08
 7.28970512e-

In [0]:
 '''
 Comment / Removal section
class Classifier():
  def __init__(self, digit):
    self.current_image = 0
    self.train_images_personal = np.zeros((10000, 28*28))
    self.train_labels_personal  = np.zeros((10000, 1))
    for i in range(len(train_images)):
      current_index = 0
      if train_labels[i] == digit:
        self.train_images_personal[current_index] = train_images[i]
        self.train_labels_personal[current_index] = train_labels[i]

  def get_and_increment(self):
    self.current_image += 1
    return self.train_images_personal[self.current_image-1], self.train_labels_personal[self.current_image-1] 
'''
